In [ ]:
#1.1. Kütüphanelerin İçe Aktarılması
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import os
from pathlib import Path

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

print("Path to dataset files:", path)

In [ ]:
#1.2. Veri Setinin Yüklenmesi

def load_dataset(dtbase_path):
  """
  SIC klasöründeki tüm görüntüleri train_df adlı bir DataFrame’e yükleyin.
  """
  image_data = []
  dataset_path = Path(dtbase_path)
  for img_path in dataset_path.rglob("*.jpg"):
    #bilgi alın
    class_name = img_path.parent.name
    file_name = img_path.name
    file_size = img_path.stat().st_size

    image_data.append({
      'file_path': str(img_path),
      'file_name': file_name,
      'class': class_name,
      'file_size_kb': file_size
      })
  train_df = pd.DataFrame(image_data) #daraframe yaratmak
  return train_df

train_df = load_dataset(path)
print(train_df.head())
print("\n" + "="*60)
print(train_df['class'].value_counts()) #distribution göster
print("Toplam görüntü sayısını", len(train_df))

In [ ]:
#1.3. Veri Özelliklerinin İncelenmesi

def analyze_image_properties(df, sample_size=100):
  """
  Görüntülerin çözünürlük, Kanal sayılarını, Dosya boyutu bilgilerini analiz edin.
  """
  sample_size = min(sample_size, len(df))
  sample_indices = range(sample_size)
  resolution = []
  channels = []
  for idx in sample_indices:
    img_path = df.iloc[idx]['file_path']
    img = cv2.imread(img_path)
    height, width, channel = img.shape
    resolution.append((height, width))
    channels.append(channel)
  return resolution, channels

resolution, channels =  analyze_image_properties(train_df, sample_size=100)

#çözünürlük analiz
if resolution:
    heights = [r[0] for r in resolution]
    widths = [r[1] for r in resolution]

    print(f"Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.2f}")
    print(f"Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.2f}")
    print(f"Most common resolution: {max(set(resolution), key=resolution.count)}")

print("\n")

#Kanal sayılarını analiz
if channels:
  unique_channels = set(channels)
  if 3 in unique_channels:
    print("RGB image detected")
  if 1 in unique_channels:
    print("grayscale image detected")

print("\n")

#Dosya boyutu analiz
print(f"Dosya boyutu (KB) min: {train_df['file_size_kb'].min()}")
print(f"Dosya boyutu (KB) max: {train_df['file_size_kb'].max()}")
print(f"Dosya boyutu (KB) mean: {train_df['file_size_kb'].mean()}")
print(f"Dosya boyutu (KB) median: {train_df['file_size_kb'].median()}")

In [ ]:
#2. Görüntü Yükleme ve Görselleştirme

#2.1. Rastgele Görüntüler Seçme

random_indices = [400, 403, 399, 800, 1002, 2021, 20, 80, 693]
random_images_df = train_df.iloc[random_indices].reset_index()
print(f"Selected {len(random_images_df)} random images:")
print(random_images_df[['file_name', 'class']])

In [ ]:
#RGB ve Grayscale sürümleri yan yana görüntüleme
fig, axes = plt.subplots(9, 2, figsize=(10, 20))
fig.suptitle('RGB (Left) vs Grayscale (Right)', fontsize=16, y=0.995)
for idx in range(9):
  img_path = random_images_df.iloc[idx]['file_path']
  #okumak RGB image
  img_rgb = cv2.imread(img_path)
  img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB) #Bgr'yi rgb'ye dönüştürme
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY) #grayscale'ye dönüştürme
  #görüntüleri görüntüleme
  axes[idx, 0].imshow(img_rgb)
  axes[idx, 0].set_title(f'RGB - {random_images_df.iloc[idx]["class"]}', fontsize=10)

  axes[idx, 1].imshow(img_gray, cmap='gray')
  axes[idx, 1].set_title(f'Grayscale - {random_images_df.iloc[idx]["class"]}', fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
#2.2. Rastgele Görüntülerin İstatistiksel Özellikleri

def calculate_statistics(img_rgb, img_gray):
  """
  değerlerini hesaplamak ve karşılaştırmak.
  """
  stats = {
      'RGB': {
          'min': img_rgb.min(),
          'max': img_rgb.max(),
          'mean': img_rgb.mean(),
          'std': img_rgb.std()
      },
      'grayscale': {
          'min': img_gray.min(),
          'max': img_gray.max(),
          'mean': img_gray.mean(),
          'std': img_gray.std()
      }
  }
  return stats

for idx in range(len(random_images_df)):
  img_path = random_images_df.iloc[idx]['file_path']
  img_rgb = cv2.imread(img_path)
  img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
  stats = calculate_statistics(img_rgb, img_gray)
  print(f"\nRGB İstatistiksel:" f"Overall - Min: {stats['RGB']['min']}, Max: {stats['RGB']['max']}, "
          f"Mean: {stats['RGB']['mean']:.2f}, Std: {stats['RGB']['std']:.2f}")
  print(f"\nGrayscale İstatistiksel:" f"Overall - Min: {stats['grayscale']['min']}, Max: {stats['grayscale']['max']},"
          f"Mean: {stats['grayscale']['mean']}, {stats['grayscale']['std']}")


In [ ]:
#2.3. Histogram Çizimi (Zorunlu – RGB + Grayscale)
fig, axes = plt.subplots(9, 2, figsize=(14, 25))
fig.suptitle('Histograms: RGB (Left) vs Grayscale (Right)', fontsize=16, y=0.995)

for idx in range(len(random_images_df)):
    img_path = random_images_df.iloc[idx]['file_path']

    #  Görüntüler oku
    img_rgb = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # RGB Histogramı
    colors = ('r', 'g', 'b')
    channel_names = ('Red', 'Green', 'Blue')

    for i, (color, name) in enumerate(zip(colors, channel_names)):
        hist = cv2.calcHist([img_rgb], [i], None, [256], [0, 256])
        axes[idx, 0].plot(hist, color=color, label=name, alpha=0.7)

    axes[idx, 0].set_xlim([0, 256])
    axes[idx, 0].set_xlabel('Pixel Intensity')
    axes[idx, 0].set_ylabel('Frequency')
    axes[idx, 0].set_title(f'RGB Histogram - Image {idx+1}', fontsize=10)
    axes[idx, 0].legend()
    axes[idx, 0].grid(True, alpha=0.3)

    # Grayscale Histogramı
    hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
    axes[idx, 1].plot(hist_gray, color='black')
    axes[idx, 1].fill_between(range(256), hist_gray.flatten(), alpha=0.3, color='gray')
    axes[idx, 1].set_xlim([0, 256])
    axes[idx, 1].set_xlabel('Pixel Intensity')
    axes[idx, 1].set_ylabel('Frequency')
    axes[idx, 1].set_title(f'Grayscale Histogram - Image {idx+1}', fontsize=10)
    axes[idx, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("""çünkü rastgele resim seçimi yapılıyor, analiz word'de görüntüleniyor
HİSTOGRAMLARIN ANALİZİ:
Image 1
RGB: Kırmızı orta–yüksek tonlarda (~150–200) daha baskın. Yeşil ve mavi orta tonlarda yoğunlaşmış.
Grayscale: Yoğunluk ~120–180 aralığında.
Yorum: Dengeli, hafif aydınlık bir görüntü.
Image 2
RGB: Bütün kanallar sağa kaymış; kırmızı, yeşil ve mavi ~200–240 aralığında yoğun. Kırmızı hafif daha baskın.
Grayscale: Ana tepe ~210–240.
Yorum: Çok parlak bir görüntü; histogram neredeyse tamamen sağ tarafta.
Image 3
RGB: Kırmızı net şekilde baskın (~180–230). Yeşil ve mavi orta tonlarda (~100–160).
Grayscale: Yoğunluk ~130–180.
Yorum: Kırmızı kanalın etkisi yüksek; gri histogram orta–aydınlık bölgede.
Image 4
RGB: Mavi ~40–100 aralığında yüksek. Yeşil ~60–120. Kırmızı ~120–180.
Grayscale: Tepe ~50–100 aralığında.
Yorum: Görüntü koyu/orta ton ağırlıklı; özellikle mavi baskın.
Image 5
RGB: Kanallar ~150–220 arasında yoğun; kırmızı sağda daha yüksek.
Grayscale: Ana tepe ~130–200.
Yorum: Aydınlık alanlar fazla, RGB kanallar dengeli ama kırmızı önde.
Image 6
RGB: Kırmızı ve mavi ~170–210 arasında yoğun; yeşil biraz daha düşük. Kırmızı pik en yüksek.
Grayscale: Tepe ~160–210.
Yorum: Aydınlık ağırlıklı; gri histogram sağ bölgede yoğun.
Image 7
RGB: Kırmızı ~150–180 aralığında çok yüksek. Yeşil ~180–220, mavi ~200–250 aralığında.
Grayscale: Yoğunluk ~160–220.
Yorum: Görüntü çok parlak; tüm kanallar yüksek parlaklığa yayılmış
Image 8
RGB: Kırmızı sağda (~210–250) baskın. Yeşil ve mavi orta tonlarda (~120–180).
Grayscale: Tepe ~150–190.
Yorum: Aydınlık ama dengeli bir sahne; kırmızı kanal öne çıkıyor.
Image 9
RGB: Yeşil en belirgin (~130–170). Mavi ~160, kırmızı ~180 civarında.
Grayscale: Ana yoğunluk ~120–170.
Yorum: Orta ton ağırlıklı; yeşil kanal baskın.
""")

In [ ]:
#3. Görüntü İşleme ve İyileştirme
#3.1. Kontrast Germe (Stretching) işlemini hem RGB hem grayscale görüntülere uygulayın.
# Select one sample image for demonstration
sample_idx = [400, 403, 399, 800, 1002, 2021, 20, 80, 693]


def contrast_stretching(image):
    """
    Apply contrast stretching to normalize pixel values to full range [0, 255]
    """
    # Find min and max pixel values
    min_val = np.min(image)
    max_val = np.max(image)

    # Apply stretching formula
    if max_val - min_val > 0:
        stretched = ((image - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    else:
        stretched = image

    return stretched

for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    # Apply contrast stretching to RGB (each channel separately)
    img_rgb_stretched = np.zeros_like(img_rgb_original)
    for channel in range(3):
        img_rgb_stretched[:,:,channel] = contrast_stretching(img_rgb_original[:,:,channel])

    # Apply contrast stretching to Grayscale
    img_gray_stretched = contrast_stretching(img_gray_original)

    # Display results for each image
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'3.1. Contrast Stretching Comparison - Image {idx}', fontsize=16, fontweight='bold')

    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_rgb_stretched)
    axes[0, 1].set_title('Stretched RGB', fontsize=12)
    axes[0, 1].axis('off')

    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_stretched, cmap='gray')
    axes[1, 1].set_title('Stretched Grayscale', fontsize=12)
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    # Show histograms before and after stretching
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    fig.suptitle(f'Histograms: Before and After Contrast Stretching - Image {idx}', fontsize=16, fontweight='bold')

    # RGB histograms - Original
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[0, 0].plot(hist, color=color, alpha=0.7, label=['Red', 'Green', 'Blue'][channel])
    axes[0, 0].set_title('Original RGB Histogram')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # RGB histograms - Stretched
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_stretched], [channel], None, [256], [0, 256])
        axes[0, 1].plot(hist, color=color, alpha=0.7, label=['Red', 'Green', 'Blue'][channel])
    axes[0, 1].set_title('Stretched RGB Histogram')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Grayscale histograms
    hist_gray_orig = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist_gray_orig, color='black')
    axes[1, 0].fill_between(range(256), hist_gray_orig.flatten(), alpha=0.3)
    axes[1, 0].set_title('Original Grayscale Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    hist_gray_stretched = cv2.calcHist([img_gray_stretched], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_gray_stretched, color='black')
    axes[1, 1].fill_between(range(256), hist_gray_stretched.flatten(), alpha=0.3)
    axes[1, 1].set_title('Stretched Grayscale Histogram')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nContrast Stretching Analysis for Image {idx}:")
    print("- Stretching expands the pixel value range to utilize full [0, 255] range")
    print("- Improves contrast by spreading out concentrated histogram values")
    print("- Useful for low-contrast skin lesion images")

In [ ]:
#3.2. Histogram Eşitleme (Equalization)

# Process each image
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    # RGB: Convert to YCrCb, equalize Y channel, convert back
    img_ycrcb = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2YCrCb)
    img_ycrcb[:,:,0] = cv2.equalizeHist(img_ycrcb[:,:,0])  # Equalize Y channel only
    img_rgb_equalized = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB)

    # Grayscale: Direct histogram equalization
    img_gray_equalized = cv2.equalizeHist(img_gray_original)

    # Display results
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'3.2. Histogram Equalization Comparison - Image {idx}', fontsize=16, fontweight='bold')

    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_rgb_equalized)
    axes[0, 1].set_title('Equalized RGB (via YCrCb)', fontsize=12)
    axes[0, 1].axis('off')

    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_equalized, cmap='gray')
    axes[1, 1].set_title('Equalized Grayscale', fontsize=12)
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    # Show histograms before and after equalization
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    fig.suptitle(f'Histograms: Before and After Equalization - Image {idx}', fontsize=16, fontweight='bold')

    # RGB histograms - Original
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[0, 0].plot(hist, color=color, alpha=0.7, label=['Red', 'Green', 'Blue'][channel])
    axes[0, 0].set_title('Original RGB Histogram')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # RGB histograms - Equalized
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_equalized], [channel], None, [256], [0, 256])
        axes[0, 1].plot(hist, color=color, alpha=0.7, label=['Red', 'Green', 'Blue'][channel])
    axes[0, 1].set_title('Equalized RGB Histogram')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Grayscale histograms
    hist_gray_orig = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist_gray_orig, color='black')
    axes[1, 0].fill_between(range(256), hist_gray_orig.flatten(), alpha=0.3)
    axes[1, 0].set_title('Original Grayscale Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    hist_gray_eq = cv2.calcHist([img_gray_equalized], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_gray_eq, color='black')
    axes[1, 1].fill_between(range(256), hist_gray_eq.flatten(), alpha=0.3)
    axes[1, 1].set_title('Equalized Grayscale Histogram')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nHistogram Equalization Analysis for Image {idx}:")
    print("- Redistributes pixel intensities to achieve uniform histogram")
    print("- For RGB: YCrCb conversion preserves color while enhancing luminance")
    print("- Enhances contrast in low-contrast regions")
    print("- May over-amplify noise in some cases")

In [ ]:
#3.3. Gamma Düzeltme
def gamma_correction(image, gamma):
    """
    Apply gamma correction to adjust brightness
    gamma < 1: brightens the image
    gamma > 1: darkens the image
    """
    # Normalize to [0, 1]
    normalized = image / 255.0
    # Apply gamma correction
    corrected = np.power(normalized, gamma)
    # Scale back to [0, 255]
    return (corrected * 255).astype(np.uint8)

# Apply gamma corrections with different values
gamma_values = [0.5, 1.0, 2.0]

# Process each image
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    # For RGB
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'3.3. Gamma Correction - RGB - Image {idx}', fontsize=16, fontweight='bold')

    for j, gamma in enumerate(gamma_values):
        img_gamma = gamma_correction(img_rgb_original, gamma)
        axes[0, j].imshow(img_gamma)
        axes[0, j].set_title(f'RGB - Gamma = {gamma}', fontsize=12)
        axes[0, j].axis('off')

        # Show histogram
        for channel, color in enumerate(['r', 'g', 'b']):
            hist = cv2.calcHist([img_gamma], [channel], None, [256], [0, 256])
            axes[1, j].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
        axes[1, j].set_title(f'Histogram (γ={gamma})')
        axes[1, j].legend()
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # For Grayscale
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'3.3. Gamma Correction - Grayscale - Image {idx}', fontsize=16, fontweight='bold')

    for j, gamma in enumerate(gamma_values):
        img_gamma = gamma_correction(img_gray_original, gamma)
        axes[0, j].imshow(img_gamma, cmap='gray')
        axes[0, j].set_title(f'Grayscale - Gamma = {gamma}', fontsize=12)
        axes[0, j].axis('off')

        # Show histogram
        hist = cv2.calcHist([img_gamma], [0], None, [256], [0, 256])
        axes[1, j].plot(hist, color='black')
        axes[1, j].fill_between(range(256), hist.flatten(), alpha=0.3)
        axes[1, j].set_title(f'Histogram (γ={gamma})')
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nGamma Correction Analysis for Image {idx}:")
    print("- Gamma = 0.5: Brightens the image, reveals details in dark regions")
    print("  * Useful for underexposed skin lesion images")
    print("  * May wash out bright regions")
    print("- Gamma = 1.0: No change (original image)")
    print("- Gamma = 2.0: Darkens the image, enhances bright regions")
    print("  * Useful for overexposed images")
    print("  * May lose details in dark regions")
    print("- Non-linear transformation that affects mid-tones more than extremes")
    print("- Important for adjusting images taken under different lighting conditions")



In [ ]:
#4. Gürültü Azaltma

# Process each image
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    #4.1. Median Blur Uygulama
    # Apply median blur with different kernel sizes
    from skimage.metrics import structural_similarity as ssim
    from skimage.metrics import peak_signal_noise_ratio as psnr

    kernel_sizes = [3, 5, 7]

    # For RGB
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'4.1. Median Blur - RGB - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=11)
    axes[0, 0].axis('off')

    # Show histogram
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[1, 0].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[1, 0].set_title('Original Histogram')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Apply median blur with different kernel sizes
    for j, ksize in enumerate(kernel_sizes, 1):
        img_median = cv2.medianBlur(img_rgb_original, ksize)

        axes[0, j].imshow(img_median)
        axes[0, j].set_title(f'Median Blur (k={ksize})', fontsize=11)
        axes[0, j].axis('off')

        # Show histogram
        for channel, color in enumerate(['r', 'g', 'b']):
            hist = cv2.calcHist([img_median], [channel], None, [256], [0, 256])
            axes[1, j].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
        axes[1, j].set_title(f'Histogram (k={ksize})')
        axes[1, j].legend()
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # For Grayscale
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'4.1. Median Blur - Grayscale - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_gray_original, cmap='gray')
    axes[0, 0].set_title('Original Grayscale', fontsize=11)
    axes[0, 0].axis('off')

    # Show histogram
    hist = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist, color='black')
    axes[1, 0].fill_between(range(256), hist.flatten(), alpha=0.3)
    axes[1, 0].set_title('Original Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    # Apply median blur with different kernel sizes
    for j, ksize in enumerate(kernel_sizes, 1):
        img_median = cv2.medianBlur(img_gray_original, ksize)

        axes[0, j].imshow(img_median, cmap='gray')
        axes[0, j].set_title(f'Median Blur (k={ksize})', fontsize=11)
        axes[0, j].axis('off')

        # Show histogram
        hist = cv2.calcHist([img_median], [0], None, [256], [0, 256])
        axes[1, j].plot(hist, color='black')
        axes[1, j].fill_between(range(256), hist.flatten(), alpha=0.3)
        axes[1, j].set_title(f'Histogram (k={ksize})')
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nMedian Blur Analysis for Image {idx}:")
    print("- Kernel size 3: Subtle smoothing, preserves most details")
    print("- Kernel size 5: Moderate smoothing, good balance")
    print("- Kernel size 7: Strong smoothing, may lose fine details")
    print("- Excellent for removing salt-and-pepper noise")
    print("- Non-linear filter that preserves edges better than linear filters")

    # =============================================================================
    # 4.2. GAUSSIAN BLUR APPLICATION
    # =============================================================================
    print("\n" + "="*60)
    print(f"4.2. GAUSSIAN BLUR - Image {idx}")
    print("="*60)

    # Apply Gaussian blur with different kernel sizes
    kernel_sizes_gauss = [(3,3), (5,5), (7,7)]
    sigma = 0  # Let OpenCV calculate sigma automatically

    # For RGB
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'4.2. Gaussian Blur - RGB - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=11)
    axes[0, 0].axis('off')

    # Show histogram
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[1, 0].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[1, 0].set_title('Original Histogram')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Apply Gaussian blur with different kernel sizes
    for j, ksize in enumerate(kernel_sizes_gauss, 1):
        img_gaussian = cv2.GaussianBlur(img_rgb_original, ksize, sigma)

        axes[0, j].imshow(img_gaussian)
        axes[0, j].set_title(f'Gaussian Blur (k={ksize[0]})', fontsize=11)
        axes[0, j].axis('off')

        # Show histogram
        for channel, color in enumerate(['r', 'g', 'b']):
            hist = cv2.calcHist([img_gaussian], [channel], None, [256], [0, 256])
            axes[1, j].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
        axes[1, j].set_title(f'Histogram (k={ksize[0]})')
        axes[1, j].legend()
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # For Grayscale
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'4.2. Gaussian Blur - Grayscale - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_gray_original, cmap='gray')
    axes[0, 0].set_title('Original Grayscale', fontsize=11)
    axes[0, 0].axis('off')

    # Show histogram
    hist = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist, color='black')
    axes[1, 0].fill_between(range(256), hist.flatten(), alpha=0.3)
    axes[1, 0].set_title('Original Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    # Apply Gaussian blur with different kernel sizes
    for j, ksize in enumerate(kernel_sizes_gauss, 1):
        img_gaussian = cv2.GaussianBlur(img_gray_original, ksize, sigma)

        axes[0, j].imshow(img_gaussian, cmap='gray')
        axes[0, j].set_title(f'Gaussian Blur (k={ksize[0]})', fontsize=11)
        axes[0, j].axis('off')

        # Show histogram
        hist = cv2.calcHist([img_gaussian], [0], None, [256], [0, 256])
        axes[1, j].plot(hist, color='black')
        axes[1, j].fill_between(range(256), hist.flatten(), alpha=0.3)
        axes[1, j].set_title(f'Histogram (k={ksize[0]})')
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nGaussian Blur Analysis for Image {idx}:")
    print("- Kernel size 3: Light smoothing with minimal blur")
    print("- Kernel size 5: Moderate smoothing, commonly used")
    print("- Kernel size 7: Heavy smoothing, significant blur")
    print("- Weighted average based on Gaussian distribution")
    print("- Linear filter that smooths uniformly, including edges")
    print("- Better for reducing Gaussian noise")

    # =============================================================================
    # COMPARISON: MEDIAN vs GAUSSIAN BLUR
    # =============================================================================
    print("\n" + "="*60)
    print(f"COMPARISON: MEDIAN vs GAUSSIAN BLUR - Image {idx}")
    print("="*60)

    # Use kernel size 5 for fair comparison
    median_rgb = cv2.medianBlur(img_rgb_original, 5)
    gaussian_rgb = cv2.GaussianBlur(img_rgb_original, (5,5), 0)

    median_gray = cv2.medianBlur(img_gray_original, 5)
    gaussian_gray = cv2.GaussianBlur(img_gray_original, (5,5), 0)

    # Display comparison
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'Comparison: Median vs Gaussian Blur (kernel=5) - Image {idx}', fontsize=16, fontweight='bold')

    # RGB comparison
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(median_rgb)
    axes[0, 1].set_title('Median Blur RGB', fontsize=12)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(gaussian_rgb)
    axes[0, 2].set_title('Gaussian Blur RGB', fontsize=12)
    axes[0, 2].axis('off')

    # Grayscale comparison
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(median_gray, cmap='gray')
    axes[1, 1].set_title('Median Blur Grayscale', fontsize=12)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(gaussian_gray, cmap='gray')
    axes[1, 2].set_title('Gaussian Blur Grayscale', fontsize=12)
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Edge detection to compare edge preservation
    print(f"\nEdge Preservation Analysis for Image {idx} (using Canny edge detection):")

    # Apply Canny edge detection
    edges_original = cv2.Canny(img_gray_original, 100, 200)
    edges_median = cv2.Canny(median_gray, 100, 200)
    edges_gaussian = cv2.Canny(gaussian_gray, 100, 200)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'Edge Detection: Comparing Edge Preservation - Image {idx}', fontsize=16, fontweight='bold')

    axes[0].imshow(edges_original, cmap='gray')
    axes[0].set_title(f'Original Edges\n({np.count_nonzero(edges_original)} edge pixels)', fontsize=11)
    axes[0].axis('off')

    axes[1].imshow(edges_median, cmap='gray')
    axes[1].set_title(f'Median Blur Edges\n({np.count_nonzero(edges_median)} edge pixels)', fontsize=11)
    axes[1].axis('off')

    axes[2].imshow(edges_gaussian, cmap='gray')
    axes[2].set_title(f'Gaussian Blur Edges\n({np.count_nonzero(edges_gaussian)} edge pixels)', fontsize=11)
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    edge_loss_median = (np.count_nonzero(edges_original) - np.count_nonzero(edges_median)) / np.count_nonzero(edges_original) * 100
    edge_loss_gaussian = (np.count_nonzero(edges_original) - np.count_nonzero(edges_gaussian)) / np.count_nonzero(edges_original) * 100

    print(f"\nEdge pixel loss for Image {idx}:")
    print(f"- Median Blur: {edge_loss_median:.2f}% edge pixels lost")
    print(f"- Gaussian Blur: {edge_loss_gaussian:.2f}% edge pixels lost")

    # Calculate quality metrics
    print("\n" + "="*60)
    print(f"QUALITY METRICS COMPARISON - Image {idx}")
    print("="*60)

    def calculate_metrics(original, filtered, filter_name):
        """Calculate PSNR and SSIM metrics"""
        # For RGB, convert to grayscale for metrics
        if len(original.shape) == 3:
            orig_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
            filt_gray = cv2.cvtColor(filtered, cv2.COLOR_RGB2GRAY)
        else:
            orig_gray = original
            filt_gray = filtered

        psnr_value = psnr(orig_gray, filt_gray)
        ssim_value = ssim(orig_gray, filt_gray)

        return psnr_value, ssim_value

    # Calculate metrics
    psnr_median_rgb, ssim_median_rgb = calculate_metrics(img_rgb_original, median_rgb, "Median RGB")
    psnr_gaussian_rgb, ssim_gaussian_rgb = calculate_metrics(img_rgb_original, gaussian_rgb, "Gaussian RGB")
    psnr_median_gray, ssim_median_gray = calculate_metrics(img_gray_original, median_gray, "Median Gray")
    psnr_gaussian_gray, ssim_gaussian_gray = calculate_metrics(img_gray_original, gaussian_gray, "Gaussian Gray")

    # Create comparison table
    comparison_data = [
        {
            'Method': 'Median Blur RGB',
            'PSNR (dB)': f'{psnr_median_rgb:.2f}',
            'SSIM': f'{ssim_median_rgb:.4f}',
            'Edge Loss (%)': f'{edge_loss_median:.2f}'
        },
        {
            'Method': 'Gaussian Blur RGB',
            'PSNR (dB)': f'{psnr_gaussian_rgb:.2f}',
            'SSIM': f'{ssim_gaussian_rgb:.4f}',
            'Edge Loss (%)': f'{edge_loss_gaussian:.2f}'
        },
        {
            'Method': 'Median Blur Grayscale',
            'PSNR (dB)': f'{psnr_median_gray:.2f}',
            'SSIM': f'{ssim_median_gray:.4f}',
            'Edge Loss (%)': f'{edge_loss_median:.2f}'
        },
        {
            'Method': 'Gaussian Blur Grayscale',
            'PSNR (dB)': f'{psnr_gaussian_gray:.2f}',
            'SSIM': f'{ssim_gaussian_gray:.4f}',
            'Edge Loss (%)': f'{edge_loss_gaussian:.2f}'
        }
    ]

    comparison_df = pd.DataFrame(comparison_data)
    print("\n" + comparison_df.to_string(index=False))

    print("\n" + "="*80)
    print(f"COMPLETED PROCESSING FOR IMAGE {idx}")
    print("="*80)

In [ ]:
#5. Döndürme ve Ayna Çevirme (Flipping)

# Process each image
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    #5.1. Rastgele Döndürme
    def rotate_image(image, angle):
        """
        Rotate image by given angle (in degrees) around center
        Positive angle = counter-clockwise rotation
        """
        # Get image dimensions
        height, width = image.shape[:2]

        # Calculate rotation matrix
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Perform rotation
        rotated = cv2.warpAffine(image, rotation_matrix, (width, height),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_REFLECT)

        return rotated

    # Generate random angles between 0 and 10 degrees
    np.random.seed(42 + i)  # Different seed for each image
    random_angles = np.random.uniform(0, 10, 5)

    print(f"\nRandom angles generated for Image {idx}:")
    for j, angle in enumerate(random_angles, 1):
        print(f"  Angle {j}: {angle:.2f}°")

    # Apply rotations to RGB
    fig, axes = plt.subplots(2, 6, figsize=(18, 7))
    fig.suptitle(f'5.1. Random Rotation - RGB Images (0° to 10°) - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original\n(0°)', fontsize=10)
    axes[0, 0].axis('off')

    # Show histogram
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[1, 0].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[1, 0].set_title('Histogram')
    axes[1, 0].legend(fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)

    # Apply rotations
    for j, angle in enumerate(random_angles, 1):
        img_rotated = rotate_image(img_rgb_original, angle)

        axes[0, j].imshow(img_rotated)
        axes[0, j].set_title(f'Rotated\n({angle:.2f}°)', fontsize=10)
        axes[0, j].axis('off')

        # Show histogram
        for channel, color in enumerate(['r', 'g', 'b']):
            hist = cv2.calcHist([img_rotated], [channel], None, [256], [0, 256])
            axes[1, j].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
        axes[1, j].set_title(f'Histogram ({angle:.2f}°)', fontsize=9)
        axes[1, j].legend(fontsize=8)
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Apply rotations to Grayscale
    fig, axes = plt.subplots(2, 6, figsize=(18, 7))
    fig.suptitle(f'5.1. Random Rotation - Grayscale Images (0° to 10°) - Image {idx}', fontsize=16, fontweight='bold')

    # Original
    axes[0, 0].imshow(img_gray_original, cmap='gray')
    axes[0, 0].set_title('Original\n(0°)', fontsize=10)
    axes[0, 0].axis('off')

    # Show histogram
    hist = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist, color='black')
    axes[1, 0].fill_between(range(256), hist.flatten(), alpha=0.3)
    axes[1, 0].set_title('Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    # Apply rotations
    for j, angle in enumerate(random_angles, 1):
        img_rotated = rotate_image(img_gray_original, angle)

        axes[0, j].imshow(img_rotated, cmap='gray')
        axes[0, j].set_title(f'Rotated\n({angle:.2f}°)', fontsize=10)
        axes[0, j].axis('off')

        # Show histogram
        hist = cv2.calcHist([img_rotated], [0], None, [256], [0, 256])
        axes[1, j].plot(hist, color='black')
        axes[1, j].fill_between(range(256), hist.flatten(), alpha=0.3)
        axes[1, j].set_title(f'Histogram ({angle:.2f}°)', fontsize=9)
        axes[1, j].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nRotation Analysis for Image {idx}:")
    print("- Small rotations (0-10°) preserve most image information")
    print("- Border pixels are filled using BORDER_REFLECT mode")
    print("- Useful for data augmentation in machine learning")
    print("- Helps model learn rotation-invariant features")
    print("- Histogram distribution remains largely unchanged")
    print("- Important: Skin lesions can appear at any orientation")

    # Demonstrate different rotation angles for comparison
    print("\n" + "-"*60)
    print(f"COMPARISON: Different Rotation Angles - Image {idx}")
    print("-"*60)

    comparison_angles = [0, 5, 10, 45, 90, 180]

    fig, axes = plt.subplots(2, 6, figsize=(18, 7))
    fig.suptitle(f'Rotation Angle Comparison - RGB - Image {idx}', fontsize=16, fontweight='bold')

    for j, angle in enumerate(comparison_angles):
        img_rotated = rotate_image(img_rgb_original, angle)

        axes[0, j].imshow(img_rotated)
        axes[0, j].set_title(f'{angle}°', fontsize=12)
        axes[0, j].axis('off')

        # Grayscale version
        img_rotated_gray = rotate_image(img_gray_original, angle)
        axes[1, j].imshow(img_rotated_gray, cmap='gray')
        axes[1, j].set_title(f'{angle}° (Gray)', fontsize=12)
        axes[1, j].axis('off')

    plt.tight_layout()
    plt.show()

    # =============================================================================
    # 5.2. HORIZONTAL FLIPPING (MIRROR)
    # =============================================================================
    print("\n" + "="*60)
    print(f"5.2. HORIZONTAL FLIPPING - Image {idx}")
    print("="*60)

    # Apply horizontal flip
    img_rgb_flipped = cv2.flip(img_rgb_original, 1)  # 1 = horizontal flip
    img_gray_flipped = cv2.flip(img_gray_original, 1)

    # Display comparison
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'5.2. Horizontal Flipping Comparison - Image {idx}', fontsize=16, fontweight='bold')

    # RGB Original
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=12)
    axes[0, 0].axis('off')

    # RGB Flipped
    axes[0, 1].imshow(img_rgb_flipped)
    axes[0, 1].set_title('Horizontally Flipped RGB', fontsize=12)
    axes[0, 1].axis('off')

    # RGB Histogram Original
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[0, 2].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[0, 2].set_title('Original RGB Histogram')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    # RGB Histogram Flipped
    for channel, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([img_rgb_flipped], [channel], None, [256], [0, 256])
        axes[0, 3].plot(hist, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[0, 3].set_title('Flipped RGB Histogram')
    axes[0, 3].legend()
    axes[0, 3].grid(True, alpha=0.3)

    # Grayscale Original
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=12)
    axes[1, 0].axis('off')

    # Grayscale Flipped
    axes[1, 1].imshow(img_gray_flipped, cmap='gray')
    axes[1, 1].set_title('Horizontally Flipped Grayscale', fontsize=12)
    axes[1, 1].axis('off')

    # Grayscale Histogram Original
    hist = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 2].plot(hist, color='black')
    axes[1, 2].fill_between(range(256), hist.flatten(), alpha=0.3)
    axes[1, 2].set_title('Original Grayscale Histogram')
    axes[1, 2].grid(True, alpha=0.3)

    # Grayscale Histogram Flipped
    hist = cv2.calcHist([img_gray_flipped], [0], None, [256], [0, 256])
    axes[1, 3].plot(hist, color='black')
    axes[1, 3].fill_between(range(256), hist.flatten(), alpha=0.3)
    axes[1, 3].set_title('Flipped Grayscale Histogram')
    axes[1, 3].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nHorizontal Flipping Analysis for Image {idx}:")
    print("- Creates mirror image along vertical axis")
    print("- Pixel intensity distribution (histogram) remains IDENTICAL")
    print("- Spatial arrangement changes but statistical properties don't")
    print("- Important for data augmentation in medical imaging")

    # Demonstrate all flip types
    print("\n" + "-"*60)
    print(f"ALL FLIP TYPES DEMONSTRATION - Image {idx}")
    print("-"*60)

    # Different flip modes
    flip_modes = {
        'Original': None,
        'Horizontal (Left↔Right)': 1,
        'Vertical (Top↔Bottom)': 0,
        'Both (180° rotation)': -1
    }

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'All Flip Types - RGB and Grayscale - Image {idx}', fontsize=16, fontweight='bold')

    for j, (flip_name, flip_code) in enumerate(flip_modes.items()):
        if flip_code is None:
            img_rgb_flip = img_rgb_original
            img_gray_flip = img_gray_original
        else:
            img_rgb_flip = cv2.flip(img_rgb_original, flip_code)
            img_gray_flip = cv2.flip(img_gray_original, flip_code)

        # RGB
        axes[0, j].imshow(img_rgb_flip)
        axes[0, j].set_title(f'{flip_name}\n(RGB)', fontsize=10)
        axes[0, j].axis('off')

        # Grayscale
        axes[1, j].imshow(img_gray_flip, cmap='gray')
        axes[1, j].set_title(f'{flip_name}\n(Grayscale)', fontsize=10)
        axes[1, j].axis('off')

    plt.tight_layout()
    plt.show()

    # =============================================================================
    # SYMMETRY ANALYSIS
    # =============================================================================
    print("\n" + "="*60)
    print(f"SYMMETRY ANALYSIS - Image {idx}")
    print("="*60)

    def analyze_symmetry(original, flipped):
        """
        Calculate symmetry score by comparing original with flipped version
        """
        # Calculate absolute difference
        diff = np.abs(original.astype(float) - flipped.astype(float))

        # Calculate symmetry score (0 = perfect symmetry, higher = more asymmetric)
        symmetry_score = np.mean(diff)

        return diff, symmetry_score

    # Analyze RGB symmetry
    diff_rgb, symmetry_rgb = analyze_symmetry(img_rgb_original, img_rgb_flipped)

    # Analyze Grayscale symmetry
    diff_gray, symmetry_gray = analyze_symmetry(img_gray_original, img_gray_flipped)

    print(f"\nSymmetry Scores for Image {idx} (lower = more symmetric):")
    print(f"  RGB Symmetry Score: {symmetry_rgb:.2f}")
    print(f"  Grayscale Symmetry Score: {symmetry_gray:.2f}")

    # Visualize differences
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'Symmetry Analysis: Difference Maps - Image {idx}', fontsize=16, fontweight='bold')

    # RGB
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_rgb_flipped)
    axes[0, 1].set_title('Flipped RGB', fontsize=11)
    axes[0, 1].axis('off')

    # Show difference as heatmap (average across channels for RGB)
    diff_rgb_avg = np.mean(diff_rgb, axis=2)
    im = axes[0, 2].imshow(diff_rgb_avg, cmap='hot')
    axes[0, 2].set_title(f'Difference Map\n(Score: {symmetry_rgb:.2f})', fontsize=11)
    axes[0, 2].axis('off')
    plt.colorbar(im, ax=axes[0, 2], fraction=0.046)

    # Grayscale
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_flipped, cmap='gray')
    axes[1, 1].set_title('Flipped Grayscale', fontsize=11)
    axes[1, 1].axis('off')

    im = axes[1, 2].imshow(diff_gray, cmap='hot')
    axes[1, 2].set_title(f'Difference Map\n(Score: {symmetry_gray:.2f})', fontsize=11)
    axes[1, 2].axis('off')
    plt.colorbar(im, ax=axes[1, 2], fraction=0.046)

    plt.tight_layout()
    plt.show()

    print(f"\nSymmetry Observations for Image {idx}:")
    print("- Higher difference values (brighter in heatmap) indicate asymmetric regions")
    print("- Skin lesions often show asymmetry, which is a diagnostic feature")
    print("- Asymmetry is one of the 'ABCDE' criteria for melanoma detection:")
    print("  A = Asymmetry, B = Border, C = Color, D = Diameter, E = Evolution")
    print("- Flipping helps identify if lesion has consistent features on both sides")

    # =============================================================================
    # COMBINED TRANSFORMATIONS
    # =============================================================================
    print("\n" + "="*60)
    print(f"COMBINED TRANSFORMATIONS - Image {idx}")
    print("="*60)

    # Apply rotation + flip
    angle_combined = np.random.uniform(0, 10)
    img_rgb_rotated = rotate_image(img_rgb_original, angle_combined)
    img_rgb_rot_flip = cv2.flip(img_rgb_rotated, 1)

    img_gray_rotated = rotate_image(img_gray_original, angle_combined)
    img_gray_rot_flip = cv2.flip(img_gray_rotated, 1)

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'Combined Transformations: Rotation ({angle_combined:.2f}°) + Horizontal Flip - Image {idx}',
                 fontsize=16, fontweight='bold')

    # RGB transformations
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_rgb_rotated)
    axes[0, 1].set_title(f'Rotated {angle_combined:.2f}°', fontsize=11)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(img_rgb_flipped)
    axes[0, 2].set_title('Flipped', fontsize=11)
    axes[0, 2].axis('off')

    axes[0, 3].imshow(img_rgb_rot_flip)
    axes[0, 3].set_title('Rotated + Flipped', fontsize=11)
    axes[0, 3].axis('off')

    # Grayscale transformations
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_rotated, cmap='gray')
    axes[1, 1].set_title(f'Rotated {angle_combined:.2f}°', fontsize=11)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(img_gray_flipped, cmap='gray')
    axes[1, 2].set_title('Flipped', fontsize=11)
    axes[1, 2].axis('off')

    axes[1, 3].imshow(img_gray_rot_flip, cmap='gray')
    axes[1, 3].set_title('Rotated + Flipped', fontsize=11)
    axes[1, 3].axis('off')

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print(f"COMPLETED PROCESSING FOR IMAGE {idx}")
    print("="*80)

In [ ]:
#6. Frekans Alanında Filtreleme (FFT)  клауд

# Process each image from the predefined list
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    print(f"\nProcessing image {i+1}/9: Index {idx}")
    print(f"Image: {train_df.iloc[idx]['file_name']}")
    print(f"Class: {train_df.iloc[idx]['class']}")
    print(f"Image shape - RGB: {img_rgb_original.shape}, Grayscale: {img_gray_original.shape}")

    # =============================================================================
    # 6.1. FOURIER TRANSFORM
    # =============================================================================
    print("\n" + "="*60)
    print(f"6.1. FOURIER TRANSFORM - Image {idx}")
    print("="*60)

    def apply_fft(image):
        """
        Apply 2D Fourier Transform to grayscale image
        Returns: fft result, magnitude spectrum, phase spectrum
        """
        # Apply FFT
        fft = np.fft.fft2(image)

        # Shift zero frequency component to center
        fft_shift = np.fft.fftshift(fft)

        # Calculate magnitude spectrum (for visualization)
        magnitude_spectrum = np.abs(fft_shift)

        # Apply log transform for better visualization
        magnitude_spectrum_log = np.log(magnitude_spectrum + 1)

        # Calculate phase spectrum
        phase_spectrum = np.angle(fft_shift)

        return fft_shift, magnitude_spectrum, magnitude_spectrum_log, phase_spectrum

    # Convert RGB to grayscale for FFT (as required)
    img_gray_for_fft = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    # Apply FFT to both images
    print("\nApplying FFT to RGB-converted grayscale image...")
    fft_rgb_gray, mag_rgb_gray, mag_log_rgb_gray, phase_rgb_gray = apply_fft(img_gray_for_fft)

    print("Applying FFT to original grayscale image...")
    fft_gray, mag_gray, mag_log_gray, phase_gray = apply_fft(img_gray_original)

    # Display original images and their frequency spectrums
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    fig.suptitle(f'6.1. Fourier Transform - Frequency Spectrum Visualization - Image {idx}',
                 fontsize=16, fontweight='bold')

    # RGB converted to grayscale
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_gray_for_fft, cmap='gray')
    axes[0, 1].set_title('RGB → Grayscale\n(for FFT)', fontsize=11)
    axes[0, 1].axis('off')

    im1 = axes[0, 2].imshow(mag_log_rgb_gray, cmap='hot')
    axes[0, 2].set_title('Magnitude Spectrum\n(log scale)', fontsize=11)
    axes[0, 2].axis('off')
    plt.colorbar(im1, ax=axes[0, 2], fraction=0.046)

    im2 = axes[0, 3].imshow(phase_rgb_gray, cmap='hsv')
    axes[0, 3].set_title('Phase Spectrum', fontsize=11)
    axes[0, 3].axis('off')
    plt.colorbar(im2, ax=axes[0, 3], fraction=0.046)

    # Original grayscale
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_original, cmap='gray')
    axes[1, 1].set_title('Grayscale\n(for FFT)', fontsize=11)
    axes[1, 1].axis('off')

    im3 = axes[1, 2].imshow(mag_log_gray, cmap='hot')
    axes[1, 2].set_title('Magnitude Spectrum\n(log scale)', fontsize=11)
    axes[1, 2].axis('off')
    plt.colorbar(im3, ax=axes[1, 2], fraction=0.046)

    im4 = axes[1, 3].imshow(phase_gray, cmap='hsv')
    axes[1, 3].set_title('Phase Spectrum', fontsize=11)
    axes[1, 3].axis('off')
    plt.colorbar(im4, ax=axes[1, 3], fraction=0.046)

    plt.tight_layout()
    plt.show()

    print(f"\nFourier Transform Analysis - Image {idx}:")
    print("- Center of spectrum = low frequencies (smooth variations)")
    print("- Edges of spectrum = high frequencies (sharp details, edges)")
    print("- Bright center indicates dominant low-frequency components")
    print("- Magnitude spectrum shows frequency distribution")
    print("- Phase spectrum contains spatial information")

    # =============================================================================
    # 6.2. LOW-PASS FILTER (ALÇAK GEÇİREN FİLTRE)
    # =============================================================================
    print("\n" + "="*60)
    print(f"6.2. LOW-PASS FILTER APPLICATION - Image {idx}")
    print("="*60)

    def create_lowpass_filter(shape, cutoff_radius):
        """
        Create circular low-pass filter mask
        White (1) in center, Black (0) at edges
        """
        rows, cols = shape
        crow, ccol = rows // 2, cols // 2  # Center

        # Create meshgrid
        x = np.arange(cols) - ccol
        y = np.arange(rows) - crow
        X, Y = np.meshgrid(x, y)

        # Calculate distance from center
        distance = np.sqrt(X**2 + Y**2)

        # Create circular mask
        mask = np.zeros((rows, cols), dtype=np.float32)
        mask[distance <= cutoff_radius] = 1.0

        return mask

    # Create low-pass filters with different radii
    radii = [30, 50, 100]

    # Apply to RGB-converted grayscale
    fig, axes = plt.subplots(len(radii), 5, figsize=(20, 12))
    fig.suptitle(f'6.2. Low-Pass Filter - RGB→Grayscale Image - Image {idx}', fontsize=16, fontweight='bold')

    for j, radius in enumerate(radii):
        # Create filter mask
        mask = create_lowpass_filter(img_gray_for_fft.shape, radius)

        # Apply mask in frequency domain
        fft_filtered = fft_rgb_gray * mask

        # Calculate filtered magnitude spectrum
        mag_filtered = np.log(np.abs(fft_filtered) + 1)

        # Inverse FFT to get spatial domain image
        fft_ishift = np.fft.ifftshift(fft_filtered)
        img_filtered = np.fft.ifft2(fft_ishift)
        img_filtered = np.abs(img_filtered)

        # Display results
        axes[j, 0].imshow(img_gray_for_fft, cmap='gray')
        axes[j, 0].set_title('Original' if j == 0 else '', fontsize=10)
        axes[j, 0].axis('off')

        axes[j, 1].imshow(mask, cmap='gray')
        axes[j, 1].set_title(f'Filter Mask\n(r={radius})' if j == 0 else f'r={radius}', fontsize=10)
        axes[j, 1].axis('off')

        axes[j, 2].imshow(mag_log_rgb_gray, cmap='hot')
        axes[j, 2].set_title('Original Spectrum' if j == 0 else '', fontsize=10)
        axes[j, 2].axis('off')

        axes[j, 3].imshow(mag_filtered, cmap='hot')
        axes[j, 3].set_title(f'Filtered Spectrum' if j == 0 else '', fontsize=10)
        axes[j, 3].axis('off')

        axes[j, 4].imshow(img_filtered, cmap='gray')
        axes[j, 4].set_title(f'Filtered Image' if j == 0 else '', fontsize=10)
        axes[j, 4].axis('off')

    plt.tight_layout()
    plt.show()

    # Apply to original grayscale
    fig, axes = plt.subplots(len(radii), 5, figsize=(20, 12))
    fig.suptitle(f'6.2. Low-Pass Filter - Original Grayscale Image - Image {idx}', fontsize=16, fontweight='bold')

    for j, radius in enumerate(radii):
        # Create filter mask
        mask = create_lowpass_filter(img_gray_original.shape, radius)

        # Apply mask in frequency domain
        fft_filtered = fft_gray * mask

        # Calculate filtered magnitude spectrum
        mag_filtered = np.log(np.abs(fft_filtered) + 1)

        # Inverse FFT to get spatial domain image
        fft_ishift = np.fft.ifftshift(fft_filtered)
        img_filtered = np.fft.ifft2(fft_ishift)
        img_filtered = np.abs(img_filtered)

        # Display results
        axes[j, 0].imshow(img_gray_original, cmap='gray')
        axes[j, 0].set_title('Original' if j == 0 else '', fontsize=10)
        axes[j, 0].axis('off')

        axes[j, 1].imshow(mask, cmap='gray')
        axes[j, 1].set_title(f'Filter Mask\n(r={radius})' if j == 0 else f'r={radius}', fontsize=10)
        axes[j, 1].axis('off')

        axes[j, 2].imshow(mag_log_gray, cmap='hot')
        axes[j, 2].set_title('Original Spectrum' if j == 0 else '', fontsize=10)
        axes[j, 2].axis('off')

        axes[j, 3].imshow(mag_filtered, cmap='hot')
        axes[j, 3].set_title(f'Filtered Spectrum' if j == 0 else '', fontsize=10)
        axes[j, 3].axis('off')

        axes[j, 4].imshow(img_filtered, cmap='gray')
        axes[j, 4].set_title(f'Filtered Image' if j == 0 else '', fontsize=10)
        axes[j, 4].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nLow-Pass Filter Analysis - Image {idx}:")
    print(f"- Radius {radii[0]}: Strong filtering, very blurry, only large structures visible")
    print(f"- Radius {radii[1]}: Moderate filtering, good balance, lesion shape preserved")
    print(f"- Radius {radii[2]}: Weak filtering, most details preserved")
    print("- Smaller radius = more blurring = removes more high frequencies")
    print("- Low-pass filter removes noise and fine details")
    print("- Useful for smoothing but loses edge information")

    # =============================================================================
    # 6.3. INVERSE FOURIER TRANSFORM (DEMONSTRATION)
    # =============================================================================
    print("\n" + "="*60)
    print(f"6.3. INVERSE FOURIER TRANSFORM - Image {idx}")
    print("="*60)

    # Select moderate radius for demonstration
    best_radius = 50

    # RGB→Grayscale path
    mask_demo = create_lowpass_filter(img_gray_for_fft.shape, best_radius)
    fft_filtered_demo = fft_rgb_gray * mask_demo

    # Inverse FFT
    fft_ishift_demo = np.fft.ifftshift(fft_filtered_demo)
    img_reconstructed_rgb = np.fft.ifft2(fft_ishift_demo)
    img_reconstructed_rgb = np.abs(img_reconstructed_rgb)

    # Original Grayscale path
    fft_filtered_gray_demo = fft_gray * mask_demo
    fft_ishift_gray_demo = np.fft.ifftshift(fft_filtered_gray_demo)
    img_reconstructed_gray = np.fft.ifft2(fft_ishift_gray_demo)
    img_reconstructed_gray = np.abs(img_reconstructed_gray)

    # Display reconstruction process
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    fig.suptitle(f'6.3. Inverse FFT - Reconstruction Process (radius={best_radius}) - Image {idx}',
                 fontsize=16, fontweight='bold')

    # RGB→Grayscale reconstruction
    axes[0, 0].imshow(img_gray_for_fft, cmap='gray')
    axes[0, 0].set_title('1. Original\n(RGB→Gray)', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(mag_log_rgb_gray, cmap='hot')
    axes[0, 1].set_title('2. FFT Spectrum', fontsize=11)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(np.log(np.abs(fft_filtered_demo) + 1), cmap='hot')
    axes[0, 2].set_title('3. Filtered Spectrum', fontsize=11)
    axes[0, 2].axis('off')

    axes[0, 3].imshow(img_reconstructed_rgb, cmap='gray')
    axes[0, 3].set_title('4. Reconstructed\n(Inverse FFT)', fontsize=11)
    axes[0, 3].axis('off')

    # Original Grayscale reconstruction
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('1. Original\n(Grayscale)', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(mag_log_gray, cmap='hot')
    axes[1, 1].set_title('2. FFT Spectrum', fontsize=11)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(np.log(np.abs(fft_filtered_gray_demo) + 1), cmap='hot')
    axes[1, 2].set_title('3. Filtered Spectrum', fontsize=11)
    axes[1, 2].axis('off')

    axes[1, 3].imshow(img_reconstructed_gray, cmap='gray')
    axes[1, 3].set_title('4. Reconstructed\n(Inverse FFT)', fontsize=11)
    axes[1, 3].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nInverse FFT Analysis - Image {idx}:")
    print("- Converts frequency domain back to spatial domain")
    print("- Filtered spectrum → smoothed spatial image")
    print("- Process is reversible (FFT ↔ IFFT)")
    print("- Filtering in frequency domain = convolution in spatial domain")

    # =============================================================================
    # 6.4. COMPARISON AND DETAILED ANALYSIS
    # =============================================================================
    print("\n" + "="*60)
    print(f"6.4. FFT COMPARISON AND ANALYSIS - Image {idx}")
    print("="*60)

    # Side-by-side comparison
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'6.4. FFT Results Comparison - Image {idx}', fontsize=16, fontweight='bold')

    # RGB→Grayscale path
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_gray_for_fft, cmap='gray')
    axes[0, 1].set_title('RGB→Grayscale', fontsize=12)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(img_reconstructed_rgb, cmap='gray')
    axes[0, 2].set_title('After FFT Low-Pass', fontsize=12)
    axes[0, 2].axis('off')

    # Original Grayscale path
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title('Original Grayscale', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_original, cmap='gray')
    axes[1, 1].set_title('Grayscale (same)', fontsize=12)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(img_reconstructed_gray, cmap='gray')
    axes[1, 2].set_title('After FFT Low-Pass', fontsize=12)
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Calculate differences and statistics
    print(f"\nStatistical Comparison - Image {idx}:")
    print("-" * 60)

    stats_data = []

    # Original images
    stats_data.append({
        'Image Type': 'RGB→Gray (Original)',
        'Mean': f'{img_gray_for_fft.mean():.2f}',
        'Std': f'{img_gray_for_fft.std():.2f}',
        'Min': f'{img_gray_for_fft.min()}',
        'Max': f'{img_gray_for_fft.max()}'
    })

    stats_data.append({
        'Image Type': 'Grayscale (Original)',
        'Mean': f'{img_gray_original.mean():.2f}',
        'Std': f'{img_gray_original.std():.2f}',
        'Min': f'{img_gray_original.min()}',
        'Max': f'{img_gray_original.max()}'
    })

    # Filtered images
    stats_data.append({
        'Image Type': 'RGB→Gray (FFT Filtered)',
        'Mean': f'{img_reconstructed_rgb.mean():.2f}',
        'Std': f'{img_reconstructed_rgb.std():.2f}',
        'Min': f'{img_reconstructed_rgb.min():.2f}',
        'Max': f'{img_reconstructed_rgb.max():.2f}'
    })

    stats_data.append({
        'Image Type': 'Grayscale (FFT Filtered)',
        'Mean': f'{img_reconstructed_gray.mean():.2f}',
        'Std': f'{img_reconstructed_gray.std():.2f}',
        'Min': f'{img_reconstructed_gray.min():.2f}',
        'Max': f'{img_reconstructed_gray.max():.2f}'
    })

    stats_df = pd.DataFrame(stats_data)
    print(stats_df.to_string(index=False))

    # Difference analysis
    diff_rgb_gray = np.abs(img_gray_for_fft.astype(float) - img_gray_original.astype(float))
    diff_filtered = np.abs(img_reconstructed_rgb - img_reconstructed_gray)

    print(f"\nDifference Analysis - Image {idx}:")
    print(f"Difference between RGB→Gray and Original Grayscale:")
    print(f"  Mean absolute difference: {diff_rgb_gray.mean():.4f}")
    print(f"  Max absolute difference: {diff_rgb_gray.max():.4f}")

    print(f"\nDifference between filtered images:")
    print(f"  Mean absolute difference: {diff_filtered.mean():.4f}")
    print(f"  Max absolute difference: {diff_filtered.max():.4f}")

    # Visualize differences
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(f'Difference Analysis - Image {idx}', fontsize=16, fontweight='bold')

    im1 = axes[0].imshow(diff_rgb_gray, cmap='hot')
    axes[0].set_title(f'Original Images Difference\n(Mean: {diff_rgb_gray.mean():.4f})', fontsize=11)
    axes[0].axis('off')
    plt.colorbar(im1, ax=axes[0], fraction=0.046)

    im2 = axes[1].imshow(diff_filtered, cmap='hot')
    axes[1].set_title(f'FFT Filtered Images Difference\n(Mean: {diff_filtered.mean():.4f})', fontsize=11)
    axes[1].axis('off')
    plt.colorbar(im2, ax=axes[1], fraction=0.046)

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print(f"COMPLETED FFT PROCESSING FOR IMAGE {idx}")
    print("="*80)

In [ ]:
#7. Keskinleştirme ve Enterpolasyon

# Process each image from the predefined list
for i, idx in enumerate(sample_idx):
    sample_img_path = train_df.iloc[idx]['file_path']

    # Load the sample image
    img_rgb_original = cv2.imread(sample_img_path)
    img_rgb_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_BGR2RGB)
    img_gray_original = cv2.cvtColor(img_rgb_original, cv2.COLOR_RGB2GRAY)

    print(f"\nProcessing image {i+1}/9: Index {idx}")
    print(f"Image: {train_df.iloc[idx]['file_name']}")
    print(f"Class: {train_df.iloc[idx]['class']}")
    print(f"Original shape - RGB: {img_rgb_original.shape}, Grayscale: {img_gray_original.shape}")

    # =============================================================================
    # 7.1. UNSHARP MASKING - SHARPENING
    # =============================================================================
    print("\n" + "="*60)
    print(f"7.1. UNSHARP MASKING - SHARPENING - Image {idx}")
    print("="*60)

    def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.5, threshold=0):
        """
        Apply unsharp masking to sharpen an image

        Parameters:
        - kernel_size: Size of Gaussian kernel
        - sigma: Standard deviation for Gaussian blur
        - amount: Strength of sharpening (1.0-2.0 typical)
        - threshold: Minimum brightness change required

        Returns:
        - Sharpened image
        """
        # Step 1: Create blurred version
        if len(image.shape) == 3:  # RGB
            blurred = cv2.GaussianBlur(image, kernel_size, sigma)
        else:  # Grayscale
            blurred = cv2.GaussianBlur(image, kernel_size, sigma)

        # Step 2: Calculate the mask (original - blurred)
        mask = cv2.subtract(image, blurred)

        # Step 3: Add weighted mask to original
        sharpened = cv2.addWeighted(image, 1.0, mask, amount, 0)

        # Step 4: Apply threshold if needed
        if threshold > 0:
            low_contrast_mask = np.absolute(mask) < threshold
            np.copyto(sharpened, image, where=low_contrast_mask)

        return sharpened, blurred, mask

    # Apply unsharp masking with different parameters
    print(f"\nApplying Unsharp Masking with different parameters for Image {idx}...")

    # Parameters to test
    params = [
        {'sigma': 1.0, 'amount': 1.0, 'name': 'Mild (σ=1.0, α=1.0)'},
        {'sigma': 1.0, 'amount': 1.5, 'name': 'Moderate (σ=1.0, α=1.5)'},
        {'sigma': 2.0, 'amount': 2.0, 'name': 'Strong (σ=2.0, α=2.0)'}
    ]

    # Apply to RGB
    fig, axes = plt.subplots(len(params) + 1, 4, figsize=(16, 16))
    fig.suptitle(f'7.1. Unsharp Masking - RGB Image - Image {idx}', fontsize=16, fontweight='bold')

    # Original row
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title('Original RGB', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].set_visible(False)
    axes[0, 2].set_visible(False)
    axes[0, 3].set_visible(False)

    # Apply different sharpening strengths
    rgb_sharpened_images = []
    for j, param in enumerate(params, 1):
        sharpened, blurred, mask = unsharp_mask(
            img_rgb_original,
            kernel_size=(5, 5),
            sigma=param['sigma'],
            amount=param['amount']
        )
        rgb_sharpened_images.append(sharpened)

        axes[j, 0].imshow(sharpened)
        axes[j, 0].set_title(f'Sharpened\n{param["name"]}', fontsize=10)
        axes[j, 0].axis('off')

        axes[j, 1].imshow(blurred)
        axes[j, 1].set_title('Blurred Version', fontsize=10)
        axes[j, 1].axis('off')

        # Normalize mask for visualization
        mask_vis = cv2.normalize(mask, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        axes[j, 2].imshow(mask_vis)
        axes[j, 2].set_title('Unsharp Mask', fontsize=10)
        axes[j, 2].axis('off')

        # Show detail comparison
        axes[j, 3].imshow(sharpened)
        axes[j, 3].set_title('Sharpened (Detail)', fontsize=10)
        axes[j, 3].axis('off')

    plt.tight_layout()
    plt.show()

    # Apply to Grayscale
    fig, axes = plt.subplots(len(params) + 1, 4, figsize=(16, 16))
    fig.suptitle(f'7.1. Unsharp Masking - Grayscale Image - Image {idx}', fontsize=16, fontweight='bold')

    # Original row
    axes[0, 0].imshow(img_gray_original, cmap='gray')
    axes[0, 0].set_title('Original Grayscale', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].set_visible(False)
    axes[0, 2].set_visible(False)
    axes[0, 3].set_visible(False)

    # Apply different sharpening strengths
    gray_sharpened_images = []
    for j, param in enumerate(params, 1):
        sharpened, blurred, mask = unsharp_mask(
            img_gray_original,
            kernel_size=(5, 5),
            sigma=param['sigma'],
            amount=param['amount']
        )
        gray_sharpened_images.append(sharpened)

        axes[j, 0].imshow(sharpened, cmap='gray')
        axes[j, 0].set_title(f'Sharpened\n{param["name"]}', fontsize=10)
        axes[j, 0].axis('off')

        axes[j, 1].imshow(blurred, cmap='gray')
        axes[j, 1].set_title('Blurred Version', fontsize=10)
        axes[j, 1].axis('off')

        axes[j, 2].imshow(mask, cmap='gray')
        axes[j, 2].set_title('Unsharp Mask', fontsize=10)
        axes[j, 2].axis('off')

        axes[j, 3].imshow(sharpened, cmap='gray')
        axes[j, 3].set_title('Sharpened (Detail)', fontsize=10)
        axes[j, 3].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nUnsharp Masking Analysis - Image {idx}:")
    print("- Mild sharpening: Subtle enhancement, natural look")
    print("- Moderate sharpening: Good balance, enhances edges well")
    print("- Strong sharpening: Maximum detail, may amplify noise")
    print("- Process: Original - Blur = Mask; Original + (Amount × Mask) = Sharpened")
    print("- Enhances edges and fine details in skin lesion images")
    print("- Important for highlighting lesion boundaries")

    # Histogram comparison
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Histogram Comparison: Original vs Sharpened (Moderate) - Image {idx}', fontsize=16, fontweight='bold')

    # RGB histograms
    for channel, color in enumerate(['r', 'g', 'b']):
        hist_orig = cv2.calcHist([img_rgb_original], [channel], None, [256], [0, 256])
        axes[0, 0].plot(hist_orig, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[0, 0].set_title('Original RGB Histogram')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    for channel, color in enumerate(['r', 'g', 'b']):
        hist_sharp = cv2.calcHist([rgb_sharpened_images[1]], [channel], None, [256], [0, 256])
        axes[0, 1].plot(hist_sharp, color=color, alpha=0.7, label=['R', 'G', 'B'][channel])
    axes[0, 1].set_title('Sharpened RGB Histogram')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Grayscale histograms
    hist_gray_orig = cv2.calcHist([img_gray_original], [0], None, [256], [0, 256])
    axes[1, 0].plot(hist_gray_orig, color='black')
    axes[1, 0].fill_between(range(256), hist_gray_orig.flatten(), alpha=0.3)
    axes[1, 0].set_title('Original Grayscale Histogram')
    axes[1, 0].grid(True, alpha=0.3)

    hist_gray_sharp = cv2.calcHist([gray_sharpened_images[1]], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_gray_sharp, color='black')
    axes[1, 1].fill_between(range(256), hist_gray_sharp.flatten(), alpha=0.3)
    axes[1, 1].set_title('Sharpened Grayscale Histogram')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # =============================================================================
    # 7.2. BICUBIC INTERPOLATION - UPSCALING
    # =============================================================================
    print("\n" + "="*60)
    print(f"7.2. BICUBIC INTERPOLATION - 2X UPSCALING - Image {idx}")
    print("="*60)

    # Use moderate sharpening for interpolation
    img_rgb_sharp = rgb_sharpened_images[1]  # Moderate sharpening
    img_gray_sharp = gray_sharpened_images[1]  # Moderate sharpening

    print(f"\nOriginal RGB shape: {img_rgb_sharp.shape}")
    print(f"Original Grayscale shape: {img_gray_sharp.shape}")

    # Calculate new dimensions (2x)
    new_height_rgb = img_rgb_sharp.shape[0] * 2
    new_width_rgb = img_rgb_sharp.shape[1] * 2
    new_height_gray = img_gray_sharp.shape[0] * 2
    new_width_gray = img_gray_sharp.shape[1] * 2

    # Apply bicubic interpolation
    img_rgb_upscaled = cv2.resize(img_rgb_sharp, (new_width_rgb, new_height_rgb),
                                   interpolation=cv2.INTER_CUBIC)
    img_gray_upscaled = cv2.resize(img_gray_sharp, (new_width_gray, new_height_gray),
                                    interpolation=cv2.INTER_CUBIC)

    print(f"\nUpscaled RGB shape: {img_rgb_upscaled.shape}")
    print(f"Upscaled Grayscale shape: {img_gray_upscaled.shape}")

    # Compare different interpolation methods
    interpolation_methods = {
        'Nearest Neighbor': cv2.INTER_NEAREST,
        'Bilinear': cv2.INTER_LINEAR,
        'Bicubic': cv2.INTER_CUBIC,
        'Lanczos': cv2.INTER_LANCZOS4
    }

    # Apply to RGB
    fig, axes = plt.subplots(2, len(interpolation_methods) + 1, figsize=(20, 8))
    fig.suptitle(f'7.2. Interpolation Methods Comparison - RGB (2x Upscaling) - Image {idx}',
                 fontsize=16, fontweight='bold')

    # Original (sharpened)
    axes[0, 0].imshow(img_rgb_sharp)
    axes[0, 0].set_title('Original\n(Sharpened)', fontsize=11)
    axes[0, 0].axis('off')

    # Crop for detail view
    h, w = img_rgb_sharp.shape[:2]
    crop_y, crop_x = h//4, w//4
    crop_size = min(h//2, w//2)

    axes[1, 0].imshow(img_rgb_sharp[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size])
    axes[1, 0].set_title('Original\n(Detail)', fontsize=11)
    axes[1, 0].axis('off')

    # Apply different interpolation methods
    for j, (name, method) in enumerate(interpolation_methods.items(), 1):
        upscaled = cv2.resize(img_rgb_sharp, (new_width_rgb, new_height_rgb),
                              interpolation=method)

        axes[0, j].imshow(upscaled)
        axes[0, j].set_title(f'{name}\n2x Upscaled', fontsize=11)
        axes[0, j].axis('off')

        # Show cropped detail
        crop_y_up, crop_x_up = crop_y*2, crop_x*2
        crop_size_up = crop_size*2
        axes[1, j].imshow(upscaled[crop_y_up:crop_y_up+crop_size_up,
                                  crop_x_up:crop_x_up+crop_size_up])
        axes[1, j].set_title(f'{name}\n(Detail)', fontsize=11)
        axes[1, j].axis('off')

    plt.tight_layout()
    plt.show()

    # Apply to Grayscale
    fig, axes = plt.subplots(2, len(interpolation_methods) + 1, figsize=(20, 8))
    fig.suptitle(f'7.2. Interpolation Methods Comparison - Grayscale (2x Upscaling) - Image {idx}',
                 fontsize=16, fontweight='bold')

    # Original (sharpened)
    axes[0, 0].imshow(img_gray_sharp, cmap='gray')
    axes[0, 0].set_title('Original\n(Sharpened)', fontsize=11)
    axes[0, 0].axis('off')

    axes[1, 0].imshow(img_gray_sharp[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size], cmap='gray')
    axes[1, 0].set_title('Original\n(Detail)', fontsize=11)
    axes[1, 0].axis('off')

    # Apply different interpolation methods
    for j, (name, method) in enumerate(interpolation_methods.items(), 1):
        upscaled = cv2.resize(img_gray_sharp, (new_width_gray, new_height_gray),
                              interpolation=method)

        axes[0, j].imshow(upscaled, cmap='gray')
        axes[0, j].set_title(f'{name}\n2x Upscaled', fontsize=11)
        axes[0, j].axis('off')

        # Show cropped detail
        axes[1, j].imshow(upscaled[crop_y_up:crop_y_up+crop_size_up,
                                  crop_x_up:crop_x_up+crop_size_up], cmap='gray')
        axes[1, j].set_title(f'{name}\n(Detail)', fontsize=11)
        axes[1, j].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"\nInterpolation Methods Analysis - Image {idx}:")
    print("- Nearest Neighbor: Fastest, blocky artifacts, preserves sharp edges")
    print("- Bilinear: Fast, smooth but slightly blurry")
    print("- Bicubic: Good balance, smoother than bilinear, slower")
    print("- Lanczos: Best quality, sharpest, slowest, may have ringing")
    print("- Bicubic recommended for medical images - good quality/speed tradeoff")

    # =============================================================================
    # QUALITY ASSESSMENT
    # =============================================================================
    print("\n" + "="*60)
    print(f"QUALITY ASSESSMENT AFTER INTERPOLATION - Image {idx}")
    print("="*60)

    # Calculate edge density (higher = more detail preserved)
    def calculate_edge_density(image):
        """Calculate edge density using Canny edge detection"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
        edges = cv2.Canny(gray, 100, 200)
        edge_density = np.count_nonzero(edges) / edges.size
        return edge_density, edges

    # Original sharpened images
    edge_density_rgb_orig, edges_rgb_orig = calculate_edge_density(img_rgb_sharp)
    edge_density_gray_orig, edges_gray_orig = calculate_edge_density(img_gray_sharp)

    # Upscaled images (bicubic)
    edge_density_rgb_up, edges_rgb_up = calculate_edge_density(img_rgb_upscaled)
    edge_density_gray_up, edges_gray_up = calculate_edge_density(img_gray_upscaled)

    print(f"\nEdge Density Analysis - Image {idx}:")
    print(f"RGB Original (Sharpened): {edge_density_rgb_orig:.4f}")
    print(f"RGB Upscaled (Bicubic): {edge_density_rgb_up:.4f}")
    print(f"Grayscale Original (Sharpened): {edge_density_gray_orig:.4f}")
    print(f"Grayscale Upscaled (Bicubic): {edge_density_gray_up:.4f}")

    # Visualize edge detection
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'Edge Detection: Quality Assessment - Image {idx}', fontsize=16, fontweight='bold')

    axes[0, 0].imshow(edges_rgb_orig, cmap='gray')
    axes[0, 0].set_title(f'RGB Original Edges\nDensity: {edge_density_rgb_orig:.4f}', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(edges_rgb_up, cmap='gray')
    axes[0, 1].set_title(f'RGB Upscaled Edges\nDensity: {edge_density_rgb_up:.4f}', fontsize=11)
    axes[0, 1].axis('off')

    axes[1, 0].imshow(edges_gray_orig, cmap='gray')
    axes[1, 0].set_title(f'Grayscale Original Edges\nDensity: {edge_density_gray_orig:.4f}', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(edges_gray_up, cmap='gray')
    axes[1, 1].set_title(f'Grayscale Upscaled Edges\nDensity: {edge_density_gray_up:.4f}', fontsize=11)
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    # =============================================================================
    # FINAL COMPARISON: ORIGINAL → SHARPENED → UPSCALED
    # =============================================================================
    print("\n" + "="*60)
    print(f"COMPLETE PIPELINE VISUALIZATION - Image {idx}")
    print("="*60)

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'Complete Processing Pipeline: Original → Sharpened → Upscaled - Image {idx}',
                 fontsize=16, fontweight='bold')

    # RGB pipeline
    axes[0, 0].imshow(img_rgb_original)
    axes[0, 0].set_title(f'1. Original RGB\n{img_rgb_original.shape[:2]}', fontsize=11)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(img_rgb_sharp)
    axes[0, 1].set_title(f'2. Sharpened RGB\n{img_rgb_sharp.shape[:2]}', fontsize=11)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(img_rgb_upscaled)
    axes[0, 2].set_title(f'3. Upscaled RGB (2x)\n{img_rgb_upscaled.shape[:2]}', fontsize=11)
    axes[0, 2].axis('off')

    # Grayscale pipeline
    axes[1, 0].imshow(img_gray_original, cmap='gray')
    axes[1, 0].set_title(f'1. Original Grayscale\n{img_gray_original.shape}', fontsize=11)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(img_gray_sharp, cmap='gray')
    axes[1, 1].set_title(f'2. Sharpened Grayscale\n{img_gray_sharp.shape}', fontsize=11)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(img_gray_upscaled, cmap='gray')
    axes[1, 2].set_title(f'3. Upscaled Grayscale (2x)\n{img_gray_upscaled.shape}', fontsize=11)
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Statistical summary
    print("\n" + "="*60)
    print(f"STATISTICAL SUMMARY - Image {idx}")
    print("="*60)

    summary_data = [
        {
            'Stage': 'Original RGB',
            'Shape': str(img_rgb_original.shape[:2]),
            'Mean': f'{img_rgb_original.mean():.2f}',
            'Std': f'{img_rgb_original.std():.2f}',
            'Size (pixels)': f'{img_rgb_original.shape[0] * img_rgb_original.shape[1]:,}'
        },
        {
            'Stage': 'Sharpened RGB',
            'Shape': str(img_rgb_sharp.shape[:2]),
            'Mean': f'{img_rgb_sharp.mean():.2f}',
            'Std': f'{img_rgb_sharp.std():.2f}',
            'Size (pixels)': f'{img_rgb_sharp.shape[0] * img_rgb_sharp.shape[1]:,}'
        },
        {
            'Stage': 'Upscaled RGB (2x)',
            'Shape': str(img_rgb_upscaled.shape[:2]),
            'Mean': f'{img_rgb_upscaled.mean():.2f}',
            'Std': f'{img_rgb_upscaled.std():.2f}',
            'Size (pixels)': f'{img_rgb_upscaled.shape[0] * img_rgb_upscaled.shape[1]:,}'
        },
        {
            'Stage': 'Original Grayscale',
            'Shape': str(img_gray_original.shape),
            'Mean': f'{img_gray_original.mean():.2f}',
            'Std': f'{img_gray_original.std():.2f}',
            'Size (pixels)': f'{img_gray_original.shape[0] * img_gray_original.shape[1]:,}'
        },
        {
            'Stage': 'Sharpened Grayscale',
            'Shape': str(img_gray_sharp.shape),
            'Mean': f'{img_gray_sharp.mean():.2f}',
            'Std': f'{img_gray_sharp.std():.2f}',
            'Size (pixels)': f'{img_gray_sharp.shape[0] * img_gray_sharp.shape[1]:,}'
        },
        {
            'Stage': 'Upscaled Grayscale (2x)',
            'Shape': str(img_gray_upscaled.shape),
            'Mean': f'{img_gray_upscaled.mean():.2f}',
            'Std': f'{img_gray_upscaled.std():.2f}',
            'Size (pixels)': f'{img_gray_upscaled.shape[0] * img_gray_upscaled.shape[1]:,}'
        }
    ]

    summary_df = pd.DataFrame(summary_data)
    print("\n" + summary_df.to_string(index=False))

    print("\n" + "="*80)
    print(f"COMPLETED SHARPENING AND INTERPOLATION PROCESSING FOR IMAGE {idx}")
    print("="*80)